# Vector similarity search using Neon Postgres with pgvector and OpenAI

This notebook guides you through using [Neon Serverless Postgres](https://neon.tech/) as a vector database for OpenAI embeddings. It demonstrates how to:

1. Use embeddings created by OpenAI API
2. Store embeddings in a Neon Serverless Postgres database
3. Convert a raw text query to an embedding with OpenAI API
4. Use Neon with the `pgvector` extension to perform vector similarity search

## Prerequisites

Before you begin, ensure that you have the following:

1. A Neon Postgres database. You can create an account and set up a project with a ready-to-use `neondb` database in a few simple steps. For instructions, see [Sign up](https://neon.tech/docs/get-started-with-neon/signing-up) and [Create your first project](https://neon.tech/docs/get-started-with-neon/setting-up-a-project).
2. A connection string for your Neon database. You can copy it from the **Connection Details** widget on the Neon **Dashboard**. See [Connect from any application](https://neon.tech/docs/connect/connect-from-any-app).
3. The `pgvector` extension. Install the extension in Neon by running `CREATE EXTENSION vector;`. For instructions, see [Enable the pgvector extension](https://neon.tech/docs/extensions/pgvector#enable-the-pgvector-extension). 
4. Your [OpenAI API key](https://platform.openai.com/account/api-keys).
5. Python and `pip`.

### Install required modules

This notebook requires the `openai`, `psycopg2`, `pandas`, `wget`, and `python-dotenv` packages. You can install them with `pip`:


In [1]:
! pip install openai psycopg2 pandas wget python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 3.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.7-cp311-cp311-macosx_13_0_arm64.whl size=145725 sha256=6504c3b3cd428e1fc64e2e4de815bc360b47553ef7aa1e3aeedbf7cd59a92bc3
  Stored in directory: /Users/betuloreilly/Library/Caches/pip/wheels/ca/a2/1f/9f85470e2065e7a4089baa6c47cfa57a2f8f8e78e07390c5b4
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=8b3f18262fa8a0aa7f77783115d8a774ab59bf2ab3b388d9f6eda29bc93dc6d3
  Stored in directory: /Users/betuloreilly/Library/Caches/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built psycopg2 wget


### Prepare your OpenAI API key

An OpenAI API key is required to generate vectors for documents and queries.

If you do not have an OpenAI API key, obtain one from https://platform.openai.com/account/api-keys.

Add the OpenAI API key as an operating system environment variable. Name the variable `OPENAI_API_KEY`.

For information about configuring your OpenAI API key as an environment variable, refer to [Best Practices for API Key Safety](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety).

### Test your OpenAPI key

In [1]:
# Test to ensure that your OpenAI API key is defined as an environment variable
# If you run this notebook locally, you may have to reload the terminal and the notebook to make the environment available

import os

if os.getenv("OPENAI_API_KEY") is not None:
    print("Your OPENAI_API_KEY is ready")
else:
    print("Your OPENAI_API_KEY environment variable was not found")

Your OPENAI_API_KEY is ready


## Connect to your Neon database

Provide your Neon database connection string below or define it in an `.env` file using a `DATABASE_URL` variable. For information about obtaining a Neon connection string, see [Connect from any application](https://neon.tech/docs/connect/connect-from-any-app).

In [7]:
import os
import psycopg2
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# The connection string can be provided directly here.
# Replace the next line with Your Neon connection string.
connection_string = ""

# If connection_string is not directly provided above, 
# then check if DATABASE_URL is set in the environment or .env.
if not connection_string:
    connection_string = os.environ.get("DATABASE_URL")

    # If neither method provides a connection string, raise an error.
    if not connection_string:
        raise ValueError("Please provide a valid connection string either in the code or in the .env file as DATABASE_URL.")

# Connect using the connection string
connection = psycopg2.connect(connection_string)

# Create a new cursor object
cursor = connection.cursor()

Test the connection to your database:

In [8]:
# Execute this query to test the database connection
cursor.execute("SELECT 1;")
result = cursor.fetchone()

# Check the query result
if result == (1,):
    print("Your database connection was successful!")
else:
    print("Your connection failed.")

Your database connection was successful!


## Create a table and add indexes for your vector embeddings

The vector table created in your database is called **articles**. Each object has **title** and **content** vectors. 

An index is defined on both the **title** and **content** vector columns.

In [4]:
create_table_sql = '''
CREATE TABLE IF NOT EXISTS public.articles (
    id INTEGER NOT NULL,
    url TEXT,
    title TEXT,
    content TEXT,
    title_vector vector(1536),
    content_vector vector(1536),
    vector_id INTEGER
);

ALTER TABLE public.articles ADD PRIMARY KEY (id);
'''

# SQL statement for creating indexes
create_indexes_sql = '''
CREATE INDEX ON public.articles USING ivfflat (content_vector) WITH (lists = 1000);

CREATE INDEX ON public.articles USING ivfflat (title_vector) WITH (lists = 1000);
'''

# Execute the SQL statements
cursor.execute(create_table_sql)
cursor.execute(create_indexes_sql)

# Commit the changes
connection.commit()

## Load the data

Load the pre-computed vector data into your `articles` table from the `.csv` file. There are 25000 records, so expect the operation to take several minutes.

In [9]:
import io

# Path to your local CSV file
csv_file_path = '/Users/betuloreilly/vectordata/test4.csv'

# Define a generator function to process the csv file
def process_file(file_path):
    with open(file_path, 'r', encoding='ISO-8859-1') as file:
        for line in file:
            yield line

# Create a StringIO object to store the modified lines
modified_lines = io.StringIO(''.join(list(process_file(csv_file_path))))

# Create the COPY command for copy_expert
copy_command = '''
COPY public.articles (id, url, title, content, title_vector, content_vector, vector_id)
FROM STDIN WITH (FORMAT CSV, HEADER true, DELIMITER ';');
'''

# Execute the COPY command using copy_expert
cursor.copy_expert(copy_command, modified_lines)

# Commit the changes
connection.commit()

Check the number of records to ensure the data has been been loaded. There should be 25000 records.

In [10]:
# Check the size of the data
count_sql = """select count(*) from public.articles;"""
cursor.execute(count_sql)
result = cursor.fetchone()
print(f"Count:{result[0]}")

Count:40


## Search your data

After the data is stored in your Neon database, you can query the data for nearest neighbors. 

A `vector_name` parameter is provided that allows you to search based on "title" or "content".

In [11]:
def query_neon(query, collection_name, vector_name="title_vector", top_k=20):

    # Create an embedding vector from the user query
    embedded_query = openai.Embedding.create(
        input=query,
        model="text-embedding-ada-002",
    )["data"][0]["embedding"]

    # Convert the embedded_query to PostgreSQL compatible format
    embedded_query_pg = "[" + ",".join(map(str, embedded_query)) + "]"

    # Create the SQL query
    query_sql = f"""
    SELECT id, url, title, l2_distance({vector_name},'{embedded_query_pg}'::VECTOR(1536)) AS similarity
    FROM {collection_name}
    ORDER BY {vector_name} <-> '{embedded_query_pg}'::VECTOR(1536)
    LIMIT {top_k};
    """
    # Execute the query
    cursor.execute(query_sql)
    results = cursor.fetchall()

    return results

Run a similarity search based on `title_vector` embeddings:

In [12]:
# Query based on `title_vector` embeddings
import openai

query_results = query_neon("Greek mythology", "Articles")
for i, result in enumerate(query_results):
    print(f"{i + 1}. {result[2]} (Score: {round(1 - result[3], 3)})")

1. Astronomy (Score: 0.412)
2. Anatomy (Score: 0.409)
3. Archaeology (Score: 0.404)
4. Art (Score: 0.391)
5. Asteroid (Score: 0.382)
6. Abrahamic religion (Score: 0.371)
7. Algebra (Score: 0.369)
8. Architecture (Score: 0.359)
9. Native American (Score: 0.359)
10. Farming (Score: 0.356)
11. Australia (Score: 0.353)
12. Application (Score: 0.35)
13. Armenia (Score: 0.342)
14. Animal (Score: 0.341)
15. Atom (Score: 0.339)
16. Arithmetic (Score: 0.338)
17. April (Score: 0.335)
18. Alan Turing (Score: 0.33)
19. August (Score: 0.329)
20. Austria (Score: 0.329)


Run a similarity search based on `content_vector` embeddings:

In [13]:
# Query based on `content_vector` embeddings
query_results = query_neon("Famous battles in Greek history", "Articles", "content_vector")
for i, result in enumerate(query_results):
    print(f"{i + 1}. {result[2]} (Score: {round(1 - result[3], 3)})")

1. Astronomy (Score: 0.277)
2. Angel (Score: 0.265)
3. Australia (Score: 0.262)
4. A (Score: 0.257)
5. April (Score: 0.256)
6. August (Score: 0.254)
7. Atom (Score: 0.251)
8. Ad hominem (Score: 0.25)
9. Archaeology (Score: 0.248)
10. Architecture (Score: 0.247)
11. Alan Turing (Score: 0.243)
12. Austria (Score: 0.239)
13. Afghanistan (Score: 0.236)
14. Armenia (Score: 0.235)
15. Farming (Score: 0.235)
16. Argentina (Score: 0.234)
17. Autonomous communities of Spain (Score: 0.234)
18. Abbreviation (Score: 0.232)
19. Anatomy (Score: 0.232)
20. Algebra (Score: 0.231)


In [14]:
# Query based on `title_vector` embeddings
import openai

query_results = query_neon("Arithmetic", "Articles")
for i, result in enumerate(query_results):
    print(f"{i + 1}. {result[2]} (Score: {round(1 - result[3], 3)})")

1. Arithmetic (Score: 0.998)
2. Algebra (Score: 0.58)
3. Addition (Score: 0.453)
4. Astronomy (Score: 0.42)
5. Farming (Score: 0.418)
6. Art (Score: 0.417)
7. Application (Score: 0.41)
8. Anatomy (Score: 0.405)
9. Architecture (Score: 0.388)
10. Acceleration (Score: 0.37)
11. Atom (Score: 0.363)
12. A (Score: 0.359)
13. Alan Turing (Score: 0.359)
14. Archaeology (Score: 0.358)
15. Armenia (Score: 0.355)
16. Ad hominem (Score: 0.355)
17. August (Score: 0.354)
18. Adobe Illustrator (Score: 0.345)
19. Abbreviation (Score: 0.345)
20. Abrahamic religion (Score: 0.343)
